In [9]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on 
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  2763k      0 --:--:-- --:--:-- --:--:-- 2768k


In [10]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [11]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [29]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [12]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [13]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [15]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [30]:
import torch
import numpy as np
import tqdm
from torch.autograd import Variable
# from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [17]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [27]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.L1Loss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0157, 0.0443, 0.0205,  ..., 0.0329, 0.0176, 0.0460],
        [0.0354, 0.0321, 0.0349,  ..., 0.0123, 0.0071, 0.0300],
        [0.0129, 0.0418, 0.0460,  ..., 0.0114, 0.0298, 0.0440],
        ...,
        [0.0474, 0.0296, 0.0021,  ..., 0.0006, 0.0215, 0.0092],
        [0.0132, 0.0163, 0.0220,  ..., 0.0389, 0.0023, 0.0323],
        [0.0075, 0.0070, 0.0412,  ..., 0.0310, 0.0091, 0.0304]])
item_factors.weight tensor([[0.0061, 0.0052, 0.0383,  ..., 0.0438, 0.0113, 0.0401],
        [0.0291, 0.0442, 0.0273,  ..., 0.0122, 0.0384, 0.0342],
        [0.0466, 0.0096, 0.0119,  ..., 0.0467, 0.0036, 0.0047],
        ...,
        [0.0392, 0.0203, 0.0210,  ..., 0.0417, 0.0128, 0.0281],
        [0.0123, 0.0334, 0.0082,  ..., 0.0403, 0.0180, 0.0237],
        [0.0432, 0.0098, 0.0236,  ..., 0.0478, 0.0050, 0.0478]])


In [19]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[0.0240, 0.0208, 0.0015,  ..., 0.0290, 0.0342, 0.0250],
        [0.0182, 0.0303, 0.0464,  ..., 0.0470, 0.0324, 0.0193],
        [0.0182, 0.0443, 0.0461,  ..., 0.0448, 0.0386, 0.0359],
        ...,
        [0.0483, 0.0086, 0.0165,  ..., 0.0319, 0.0417, 0.0281],
        [0.0179, 0.0366, 0.0099,  ..., 0.0067, 0.0471, 0.0407],
        [0.0318, 0.0126, 0.0393,  ..., 0.0489, 0.0071, 0.0190]])
item_factors.weight tensor([[0.0206, 0.0167, 0.0069,  ..., 0.0126, 0.0072, 0.0336],
        [0.0477, 0.0165, 0.0050,  ..., 0.0336, 0.0027, 0.0286],
        [0.0147, 0.0456, 0.0153,  ..., 0.0125, 0.0216, 0.0126],
        ...,
        [0.0216, 0.0269, 0.0036,  ..., 0.0436, 0.0079, 0.0125],
        [0.0358, 0.0332, 0.0400,  ..., 0.0457, 0.0103, 0.0474],
        [0.0300, 0.0476, 0.0247,  ..., 0.0234, 0.0338, 0.0403]])


In [20]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [21]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [22]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

/Users/brandonmukadziwashe/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [28]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
     # Check how many ratings this movie has rat_count = ratings_df.loc[ratings_df['movieId'] == movid].count().iloc[0]
    rat_count = ratings_df.loc[ratings_df['movieId'] == movid].count().iloc[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Schindler's List (1993)
	 Usual Suspects, The (1995)
	 Back to the Future (1985)
	 Beauty and the Beast (1991)
	 Die Hard (1988)
	 Die Hard: With a Vengeance (1995)
	 Titanic (1997)
	 Batman Forever (1995)
	 Pretty Woman (1990)
	 Terminator, The (1984)
Cluster #1
	 Silence of the Lambs, The (1991)
	 Jurassic Park (1993)
	 Terminator 2: Judgment Day (1991)
	 Fight Club (1999)
	 Saving Private Ryan (1998)
	 Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
	 Ace Ventura: Pet Detective (1994)
	 Memento (2000)
	 Léon: The Professional (a.k.a. The Professional) (Léon) (1994)
	 Eternal Sunshine of the Spotless Mind (2004)
Cluster #2
	 Forrest Gump (1994)
	 Speed (1994)
	 Mask, The (1994)
	 Reservoir Dogs (1992)
	 Ocean's Eleven (2001)
	 Clear and Present Danger (1994)
	 Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)
	 Sleepless in Seattle (1993)
	 Heat (1995)
	 Austin Powers: International Man of Mystery (1997)
Cluster #3
	 Pulp Fiction (1994)
	 Gladiator (2000)
	 Mrs. Doubtfir